
# Nouran Mohamed Elsharkawy - 3202220249
# Abdallah Sherif Tera - 320220239


In [6]:
import pandas as pd


data = []

with open('/kaggle/input/wisdmdataset2021/WISDM-Dataset-2021/data/WISDM_ar_v1.1_raw.txt', 'r') as file:
    for line in file:
        line = line.strip()
        if line:
            parts = line.split(',')
            if len(parts) >= 6:
                user = parts[0]
                activity = parts[1]
                timestamp = parts[2]
                x = parts[3]
                y = parts[4]
                z = parts[5]
                

                if user and activity and timestamp and x and y and z:
                    data.append([user, activity, timestamp, x, y, z])

columns = ['user', 'activity', 'timestamp', 'x', 'y', 'z']
df = pd.DataFrame(data, columns=columns)

df['user'] = df['user'].astype(int)
df['timestamp'] = df['timestamp'].astype(float)
df['x'] = df['x'].astype(float)
df['y'] = df['y'].astype(float)
df['z'] = df['z'].astype(float)

print(df.head())


   user activity     timestamp         x          y         z
0    33  Jogging  4.910596e+13 -0.694638  12.680544  0.503953
1    33  Jogging  4.910606e+13  5.012288  11.264028  0.953424
2    33  Jogging  4.910611e+13  4.903325  10.882658 -0.081722
3    33  Jogging  4.910622e+13 -0.612916  18.496431  3.023717
4    33  Jogging  4.910633e+13 -1.184970  12.108489  7.205164


In [8]:
sample_df = df.sample(n=100)

print(sample_df)


        user    activity     timestamp     x      y         z
596713    24     Jogging  1.459384e+13  4.25   7.01 -2.110000
885145     3     Walking  1.018178e+14  8.39   9.19 -2.790000
931825    22     Jogging  1.290262e+12 -3.76   2.34  3.300000
314953    18    Upstairs  3.335530e+13  3.11   9.23  0.694638
428281    10    Upstairs  5.076012e+12  0.42   0.50 -0.840000
...      ...         ...           ...   ...    ...       ...
473688    26     Walking  3.746123e+11  7.63   4.86  1.230000
452057    28  Downstairs  2.730702e+12 -1.27   9.34  6.851035
968435     1  Downstairs  6.669662e+12 -8.92   4.02  1.610000
776531     8  Downstairs  1.119377e+14  0.04  10.04  3.210000
173750     6     Jogging  3.168702e+12  7.12   0.93 -4.712640

[100 rows x 6 columns]


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['x', 'y', 'z']] = scaler.fit_transform(df[['x', 'y', 'z']])

print(df.head())


   user activity     timestamp         x         y         z
0    33  Jogging  4.910596e+13 -0.198203  0.804141  0.019539
1    33  Jogging  4.910606e+13  0.635040  0.594169  0.114083
2    33  Jogging  4.910611e+13  0.619131  0.537638 -0.103654
3    33  Jogging  4.910622e+13 -0.186271  1.666239  0.549558
4    33  Jogging  4.910633e+13 -0.269794  0.719345  1.429102


In [11]:
import numpy as np 
window_size = 50  

X = []
y = []


for activity in df['activity'].unique():
    activity_df = df[df['activity'] == activity]
    
    for i in range(0, len(activity_df) - window_size, window_size):
        window = activity_df.iloc[i:i+window_size][['x', 'y', 'z']].values
        label = activity
        X.append(window)
        y.append(label)

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)


(21960, 50, 3) (21960,)


In [12]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

print(y_categorical.shape)


2025-04-26 16:12:32.881440: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745683953.073443      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745683953.131000      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(21960, 6)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)

print(X_train.shape, X_test.shape)


(17568, 50, 3) (4392, 50, 3)


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, input_shape=(window_size, 3), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


2025-04-26 16:14:09.756651: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 64)              │          17,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,022 (117.27 KB)

 Trainable params: 30,022 (117.27 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.5918 - loss: 1.2111 - val_accuracy: 0.7803 - val_loss: 0.6586
Epoch 2/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - accuracy: 0.7800 - loss: 0.6534 - val_accuracy: 0.7910 - val_loss: 0.5640
Epoch 3/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - accuracy: 0.8082 - loss: 0.5525 - val_accuracy: 0.8358 - val_loss: 0.4437
Epoch 4/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - accuracy: 0.8399 - loss: 0.4694 - val_accuracy: 0.8595 - val_loss: 0.4119
Epoch 5/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - accuracy: 0.8559 - loss: 0.4188 - val_accuracy: 0.8750 - val_loss: 0.3882
Epoch 6/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - accuracy: 0.8790 - loss: 0.3655 - val_accuracy: 0.8925 - val_loss: 0.3205
Epoch 7/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - accuracy: 0.8926 - loss: 0.3150 - val_accuracy: 0.8930 - val_loss: 0.3030
Epoch 8/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - accuracy: 0.9068 - loss: 0.2850 - 

In [16]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes, target_names=le.classes_))


138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step
              precision    recall  f1-score   support

  Downstairs       0.88      0.88      0.88       402
     Jogging       0.98      0.99      0.98      1369
     Sitting       0.99      0.97      0.98       240
    Standing       0.99      1.00      1.00       193
    Upstairs       0.88      0.88      0.88       491
     Walking       0.98      0.98      0.98      1697

    accuracy                           0.96      4392
   macro avg       0.95      0.95      0.95      4392
weighted avg       0.96      0.96      0.96      4392

